## Problem

Given a graph $G=(V,E)$ and a constraint function $g: V \to \mathbb{N},$ we would like to give the edges of $G$ directions so that every vertex $v$ has an indegree of at most $g(v).$

### Solution

We are going to use a variation of the push-relabel algorithm.

The output is either a violating set, or a feasible orientation as an in_edges array.

In [3]:
def indegree_algorithm(n, edges, g):     # inputs: number of vertices, the edges of G in edge-list form, and the function g
    
    t = [0]*n             # this will be the level function
    
    def iscorrect(indegreelist):      # decides whether a graph orienation is optimal
        for i in range(n):
            if len(indegreelist[i]) > g[i]:
                return False
        return True
    
    in_edges = [[] for i in range(n)]
    out_edges = [[] for i in range(n)]
    
    #---------------------------------------------------------------------------------
    # the greedy algorithm is starting
    
    for i in range(n):
        numofnormaledges = len(edges[i]) - len(in_edges[i]) - len(out_edges[i])     # the number of not yet oriented out-edges
        neighbors = edges[i].copy()
        nbs = neighbors.copy()
        for v in neighbors:
            if v in in_edges[i] or v in out_edges[i]:
                nbs.remove(v)
        neighbors = nbs.copy()              # here we determine the neighbors that are connected with unoriented edges
        indegree = len(in_edges[i])
        
        d = max(min(g[i]-indegree,numofnormaledges),0)         # this many edges needs to be oriented inwards
        
        assert(len(neighbors) == numofnormaledges)
        
        for k in range(d):           # we orient the first few edges inwards
            j = neighbors[k]
            in_edges[i].append(j)
            out_edges[j].append(i)
            
        for k in range(d,numofnormaledges):          # the rest is going to be oriented outwards
            j = neighbors[k]
            in_edges[j].append(i)
            out_edges[i].append(j)
            
    #--------------------------------------------------------------------------------
    
    if iscorrect(in_edges):                        # if we arrive at an optimal orientation, we are done
        print("Yeey, we found a solution: ", in_edges)
        return
    
    activity = [0]*n
    for i in range(n):
        activity[i] = len(in_edges[i]) - g[i]           # calculating the activity values
    
    def thereisanactive():      # decides whether there is an active vertex
        for i in range(n):
            if activity[i] > 0:
                return True
        return False
    
    def canpush(v):          # is there a vertex w so that vw can be pushed?
        for w in range(n):
            if w in in_edges[v] and t[w] == t[v]-1:
                return w
        return None
    
    count = 0
    while(count < 40 and thereisanactive()):
        count += 1
        max_activity = max(activity)
        v = activity.index(max_activity)          # if there is no active vertex, the loop stops; if there is, we take a maximal one
        
        w = canpush(v)
        if w is not None:             # we are doing a push move if we can, and switch the given edge
            in_edges[v].remove(w)
            out_edges[w].remove(v)
            in_edges[w].append(v)
            out_edges[v].append(w)
            activity[v] -= 1
            activity[w] += 1
        else:
            t[v] = t[v] + 1
            if t[v] == n:
                empty_levels = list(range(n)).copy()     # we search for an empty level, and return the vertices above it
                for i in range(n):
                    if t[i] in empty_levels:
                        empty_levels.remove(t[i])
                empty_level = max(empty_levels)
                violating_set = []
                for i in range(n):
                    if t[i] > empty_level:
                        violating_set.append(i)
                        
                print("We found a violating set: ", violating_set)
                return
    
    
    print("Yeey, we found a solution: ", in_edges)
    return

n = 4
edges = [[1,2],[0,2,3],[0,1,3],[1,2]]
g = [1,2,0,2]
indegree_algorithm(n,edges,g)





Yeey, we found a solution:  [[2], [2, 0], [], [2, 1]]


### The augmented path algorithm

Now, we are going to implement a different algorithm for the same problem using augmenting paths. To find such paths, we are going to employ BFS.

In [4]:
def indegree_augmented(n, edges, g):
    
    def iscorrect(indegreelist):      # decides whether a graph orienation is optimal
        for i in range(n):
            if len(indegreelist[i]) > g[i]:
                return False
        return True
    
    in_edges = [[] for i in range(n)]
    out_edges = [[] for i in range(n)]
    
    #---------------------------------------------------------------------------------
    # the greedy algorithm is starting
    
    for i in range(n):
        numofnormaledges = len(edges[i]) - len(in_edges[i]) - len(out_edges[i])      # the number of not yet oriented out-edges
        neighbors = edges[i].copy()
        nbs = neighbors.copy()
        for v in neighbors:
            if v in in_edges[i] or v in out_edges[i]:
                nbs.remove(v)
        neighbors = nbs.copy()              # here we determine the neighbors that are connected with unoriented edges
        indegree = len(in_edges[i])
        
        d = max(min(g[i]-indegree,numofnormaledges),0)         # this many edges needs to be oriented inwards
        
        assert(len(neighbors) == numofnormaledges)
        
        for k in range(d):           # we orient the first few edges inwards
            j = neighbors[k]
            in_edges[i].append(j)
            out_edges[j].append(i)
            
        for k in range(d,numofnormaledges):          # the rest is going to be oriented outwards
            j = neighbors[k]
            in_edges[j].append(i)
            out_edges[i].append(j)
            
    #--------------------------------------------------------------------------------
    
    if iscorrect(in_edges):                       # if we arrive at an optimal orientation, we are done
        print("Yeey, we found a solution: ", in_edges)
        return
    
    activity = [0]*n
    for i in range(n):
        activity[i] = len(in_edges[i]) - g[i]           # calculating the activity values
    
    def thereisanactive():      # decides whether there is an active vertex
        for i in range(n):
            if activity[i] > 0:
                return True
        return False
    
    #-----------------------------------------------------------------------------------
    
    count = 0
    while(count < 100 and thereisanactive()):
        count += 1
        max_activity = max(activity)
        v = activity.index(max_activity)          # if there is no active vertex, the loop stops; if there is, we take a maximal one
        
        #------------------------------------------------------------------------
        
        parent = [-1]*n        # initializes the BFS
        inspected = [False]*n
        Q = [v]
        found = False
        
        def invert(u,w):           # inverts and edge u,w
            in_edges[w].remove(u)
            out_edges[u].remove(w)
            in_edges[u].append(w)
            out_edges[w].append(u)
        
        def reinvert(w,par):        # inverts all the edges of the v-w path determined by 'par'
            current = w
            while True:
                new = par[current]
                assert(new != -1)
                invert(current,new)
                if new == v:
                    break
                current = new
            activity[v] -= 1
            activity[w] += 1
            
        while Q and not found:
            currentvertex = Q.pop(0)
            for w in in_edges[currentvertex]:
                if w not in Q and not inspected[w]:
                    Q.append(w)
                    parent[w] = currentvertex
                    if activity[w] < 0:
                        reinvert(w,parent)
                        found = True
                        break
                    
            inspected[currentvertex] = True
            
        if not Q:                   
            violating_set = []        
            for i in range(n):
                if inspected[i]:
                    violating_set.append(i)
            print("We found a violating set: ", violating_set)
            return
        
        print("Yeey, we found a solution: ", in_edges)
        return
            
                    
    

n = 4
edges = [[1,2],[0,2,3],[0,1,3],[1,2]]
g = [1,2,0,2]
indegree_augmented(n,edges,g)



Yeey, we found a solution:  [[2], [2, 0], [], [2, 1]]
